In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

In [2]:
# set default style
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['legend.title_fontsize'] = 14
mpl.rcParams['axes.labelsize'] = 12

# deciding on color map
cmap = cm.turbo

# style for errorplot
plot_style_dict = {'fmt' : 'o', 'capsize' : 2, 'markersize' : 3,}

# Defining Model Parameters

## 1. Problem Formulation

Imagine you spend some days developing a model describing the following biochemical process. 

<img src="model_scheme_1.png" width="250"/>

You decided to use simple mass action kinetics and derive the following ODEs:

$\frac{dY_1}{dt} = -k_1 Y_1 \cdot k_3 Y_3$

$\frac{dY_2}{dt} = k_1 Y_1 \cdot k_3 Y_3 - k_2 Y_2$

$\frac{dY_3}{dt} = k_2 Y_2$

You realize that you don't know any of the parameter values. Unfortunately, it is impossible to measure the parameters in vitro.
So you go into your lab and make a mix with 10 mM $Y_1$, 10 µM $Y_2$ and 10 µM $Y_3$ and measure the concentrations of all solutes over time, 
with the idea to adjust the model parameters, such that it reproduces the measurement results.
The measured concentrations (mean +- std) are shown below.


In [3]:
#TASK: read the csv called two_step_data.csv as pandas daraframe and call it: two_step_data

In [4]:
display(two_step_data.head(5))

NameError: name 'two_step_data' is not defined

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], c='C0', label=r'$Y_1$',
           **plot_style_dict)
ax.errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], c='C1', label=r'$Y_2$',
           **plot_style_dict)
ax.errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], c='C2', label=r'$Y_3$',
           **plot_style_dict)
ax.set_xlabel('Time, min')
ax.set_ylabel('Concentration, mM')
ax.legend()
plt.show()

Now you need to adjust the parameters. You start with a wild guess:

In [ ]:
from scipy.integrate import solve_ivp

In [ ]:
def two_step(t, y, p):
    # TASK: write the two_step model

    return [ds1, ds2, ds3]

In [ ]:
# given are the following initial conditions and parameters
two_step_y_init = [10, 0.01, 0.01]
two_step_params = {'k1': 10,
                   'k2' : 2,
                   'k3': 5}

# TASK: simulate the model from time 0 to 20. Save result points every 0.1 seconds.
# call the result object two_step_res


In [ ]:
fig, ax = plt.subplots()

ax.plot(two_step_res.t, two_step_res.y[0])
ax.plot(two_step_res.t, two_step_res.y[1])
ax.plot(two_step_res.t, two_step_res.y[2])

ax.errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], c='C0', label=r'$Y_1$',
            **plot_style_dict)
ax.errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], c='C1', label=r'$Y_2$',
            **plot_style_dict)
ax.errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], c='C2', label=r'$Y_3$',
            **plot_style_dict)
ax.set_xlabel('Time, min')
ax.set_ylabel('Concentration, mM')
ax.legend()
plt.show()

## 2. The cost function 

You realize that it might be cumbersum to find the parameters manually, especially as you don't want to trust your eye to evaluate the difference between 
the model simulation results and the data.
A friend tells you about the idea to mathematically define a score between the data and the simulation results. 
You could then try some combinations of parameters to find the parameter set with the smallest score.

Deciding you want to have a minimal quadratic distance between the measured datapoints and the simulation you define the cost function (also called score or objective function):

$$g(\vec{p}) = \sum_i (M(t_i|\vec{p}) - y_i)^2$$

where $M(t_i|\vec{p})$ is the model prediction given parameter set $\vec{p} = (k_1, k_2, k_3)^T$ at time $t_i$ and $y_i$ is the data value at time $t_i$.

In [ ]:
def array_to_k_dict(array):
    return {'k'+str(i+1):res for i,res in enumerate(array)}

def two_step_cost_function(params_arr, model_func, data):
    """Objective/score/cost function for the two step model.
    It takes parameters, the model function and data as arguments.
    Parameters and model are use to obtain a simulation result at the time points from the data."""
    params = array_to_k_dict(params_arr)
    
    y_init = [10, 0.01, 0.01]  ## use initial values form before
    res = solve_ivp(model_func, (0, 20), y_init, args=(params,), t_eval=data['Time'])  # simulate with parsed parameters and obtain time points corresponding to data points.
    
    # TASK: calculate squared distance between simulation and data for each measured species

    
    # calculate total error
    total_error = error_1.sum() + error_2.sum() + error_3.sum()
    return total_error

In [ ]:
current_cost = two_step_cost_function(list(two_step_params.values()), two_step, two_step_data)
print(f"Given the parameters {two_step_params['k1'] = },\n{two_step_params['k2'] = } and\n {two_step_params['k3'] = } result in a value of the objective function {current_cost = :0.2f}")

## 3. Minimizing the cost function

### Local optimization

### Steepest gradient descent

The most elementary gradient-based method is the __gradient descent__ algorithm, in which $\vec{p}$ is moved in the direction of the gradient $\nabla g(\vec{p})$ with a step size $\alpha$:
$$
\begin{equation}
\vec{p}_{n+1} = \vec{p}_n - \alpha \nabla g(\vec{p}_n)
\end{equation}
$$

The challange is to find an appropriate value for $\alpha$. To small values result in long computation times, to large values can result in an overshoot.

### Newton's method

One elegant way is to set the step size to the inverse of the second derivate (or Hessian ($H_g$)). Thus, when the curvature is small, the algorithm takes large steps, while in regions of high curvature the step size decreases. This is known as Newton's method. 
$$
\begin{equation}
 \vec{p}_{n+1} =  \vec{p}_n  - H_g(\vec{p})^{-1} \nabla g(\vec{p})
\end{equation}
$$

### Levenberg Marquardt

The Levenberg Marqurdt algorithm makes use of the fact, that the steepest gradient descent makes fast progress in steep section of the parameter landscape, while it struggles to converge in shallow regions. On the other hand Newton's method tends to be slow in steep regions and especially suited if close to the minimum. 
The Levenberg-Marquardt method benefits of the complementary advantages of the two methods. The Levenberg-Marquardt algorithm initiates as a gradient descent and over the course of optimization gradually switches into the Newton method.


### Nelder Mead

The simplex (or Nelder-Mead) algorithm initiates an adaptive, simplest polytope (simplex) with $m + 1$ vertices in $m$ dimensions (e.g. a triangle in two dimensions). The objective function is evaluated at all vertices and the worst (highest scored) vertex is reflected along the axis spanned by the others. If this reflection results in an improved score, the vertex is moved further in the reflection direction (expansion), otherwise the worst vertex is moved in direction of the remaining vertices (contraction). If none of the former operations results in an improved value of the objective function, all vertices are moved towards the best vertex. 

<img src="nelder_mead.png" width="500"/>


In [1]:
from scipy.optimize import minimize 
# import minimize function from scipy, it comes with a variety of local optimization algotihms

The [``minimize``](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html) function takes two required arguments. The function which return a score and an initial guess.
It further takes additional arguments which should be passed to the cost function as the tuple ``args``. In our case we want to pass the function which defines the model and the data to the cost function.
The optimization method can be defined as a string. 

In [ ]:
# TASK: in general the minimize function needs only an objective (or cost) function which return a value and 
# an array-like object of parameters, which is given to this function. 
# additional arguments to the objective can be passed via the args argument.
# FIND OUT WHAT IS MISSING BELOW:
lhs_log_best = [0.9186961 , 0.27733539, 0.37864059]
estimation_result = minimize(,
                             lhs_log_best, 
                             args=(two_step, ),
                             method='Nelder-Mead')  

The object returned by the minimization function (``estimation_result``) holds several information about the estimation process. 
Most importantly the cost and the best parameters.

In [ ]:
print(f"""The best parameters:
k1 = {estimation_result.x[0]}, 
k2 = {estimation_result.x[1]},
k3 = {estimation_result.x[2]}, 
resulting in a score of {np.round(estimation_result.fun, 2)}.""")

In [ ]:
est_params_dict = array_to_k_dict(estimation_result.x)

res = solve_ivp(two_step, (0, 20), two_step_y_init, args=(est_params_dict,), t_eval=np.arange(0, 20, 0.1))
fig, ax = plt.subplots()

ax.plot(res.t, res.y[0])
ax.plot(res.t, res.y[1])
ax.plot(res.t, res.y[2])

ax.errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], fmt='o', capsize=2, markersize=3, c='C0')
ax.errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], fmt='o', capsize=2, markersize=3, c='C1')
ax.errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], fmt='o', capsize=2, markersize=3, c='C2')

plt.show()

### Optimizing in log-space

As for the sampling it might also be beneficial to let the optimization algorithm run in log space. 
There fore one can re-define the cost function, such that it takes logarithmic parameters. In this example also the cost function was altered to take into account the uncertainty in the data (i.e. the standard deviation $\sigma$). 

$$g(\vec{p}) = \sum_i \frac{(M(t_i|\vec{p}) - y_i)^2}{\sigma_i^2}$$


In [ ]:
def two_step_cost_function_log(log_params, model_func, data):
    """Objective/score/cost function for the two step model.
    It takes parameters in log-space, the model function and data as arguments.
    Parameters and model are use to obtain a simulation result at the time points from the data."""
    y_init = [10, 0.01, 0.01]  ## use initial values form before
    
    params_arr = 10**(log_params)
    params = array_to_k_dict(params_arr)
    
    res = solve_ivp(model_func, (0, 20), y_init, args=(params,), t_eval=data['Time'])  # simulate with parsed parameters and obtain time points corresponding to data points.
    
    # calculated squred distance between simulation and data for each measured species
    # TASK: calculate the errors
    
    
    # calculate total error
    total_error = error_1.sum() + error_2.sum() + error_3.sum()
    return total_error

In [ ]:
estimation_result_log = minimize(two_step_cost_function_log, np.log10(lhs_log_best), args=(two_step, two_step_data),method='Nelder-Mead')
best_params = 10**(estimation_result_log.x)

In [ ]:
print(f"""The best parameters:
k1 = {best_params[0]}, 
k2 = {best_params[1]},
k3 = {best_params[2]}, 
resulting in a score of {np.round(estimation_result_log.fun, 2)}.""")

In [ ]:
res = solve_ivp(two_step,
                (0, 20), 
                two_step_y_init, 
                args=(array_to_k_dict(best_params),),
                t_eval=np.arange(0, 20, 0.1))

fig, ax = plt.subplots()

ax.plot(res.t, res.y[0])
ax.plot(res.t, res.y[1])
ax.plot(res.t, res.y[2])

ax.errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], fmt='o', capsize=2, markersize=3, c='C0')
ax.errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], fmt='o', capsize=2, markersize=3, c='C1')
ax.errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], fmt='o', capsize=2, markersize=3, c='C2')

plt.show()

## 4. Sampling

After a while you get tired of trying new parameters and decide that the computer should do the work. 
In order to test different parameter sets you sample some datapoints within an interval you believe obtains the best parameter set $\vec{p}^\star$.

### 4.1 Random

Imagine you barely have an idea of the magitude of the needed parameters. 
You decide that they should be somewhere in the interval $[0.01, 10]$. 
Now you sample uniformly in this interval and hope to get a good parameter set.

In [ ]:
# define parameter intervals
lower_sample_boundary = 1e-2
higher_sample_boundary = 10

n_samples = 5 # define number of samples
sample_dim = 3 # define sample dimension, i.e. number of parameters

# create normalization function for n_samples
norm = mpl.colors.Normalize(vmin=0,  vmax=n_samples)

# draw samples within interval (note that it was assumed that the interval for each parameter is identical)

# TASK: use a function from the numpy package to draw random n_samples samples which are uniformly distributed
# within the given boundaries. Take care that each sample has the right number of parameter (sample_dim)


print(random_samples)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(9, 3))

colors = np.arange(0, n_samples)

### plot parameter bounds
for axi in fig.axes:
    axi.plot([lower_sample_boundary, lower_sample_boundary], [higher_sample_boundary, lower_sample_boundary], color='k')
    axi.plot([higher_sample_boundary, lower_sample_boundary], [higher_sample_boundary, higher_sample_boundary], color='k')
    axi.plot([higher_sample_boundary, higher_sample_boundary], [lower_sample_boundary, higher_sample_boundary], color='k')
    axi.plot([lower_sample_boundary, higher_sample_boundary], [lower_sample_boundary, lower_sample_boundary], color='k')

# plot parameter sets
ax[0].scatter(random_samples[:,0], random_samples[:,1], c=colors,cmap=cmap)
ax[0].set_xlabel(r'$k_1$')
ax[0].set_ylabel(r'$k_2$')

ax[1].scatter(random_samples[:,0], random_samples[:,2], c=colors,cmap=cmap)
ax[1].set_xlabel(r'$k_1$')
ax[1].set_ylabel(r'$k_3$')

sc = ax[2].scatter(random_samples[:,1], random_samples[:,2], c=colors,cmap=cmap)
ax[2].set_xlabel(r'$k_2$')
ax[2].set_ylabel(r'$k_3$')  

## add legend, adjust tick params etc.
fig.legend(*sc.legend_elements(), title='Parameter Set', bbox_to_anchor=(1.2, 0.95))


plt.tight_layout()
plt.show()

The figure above shows your sampling results. Now you want to see how they compare to your data and you simulation results.

In [ ]:
def get_scores_results_for_samples(p_samples, data):
    
    results = []
    scores = []

    for p_vec in p_samples:

        params_v = {'k1': p_vec[0], 'k2' : p_vec[1], 'k3': p_vec[2]}

        two_step_res = solve_ivp(two_step,
                                 (0, 20), 
                                 two_step_y_init, 
                                 args=(params_v,), 
                                 t_eval=np.arange(0, 20, 0.1))
        results.append(two_step_res)
        score = two_step_cost_function(p_vec, two_step, data)
        scores.append(score)
    return(scores, results)    

In [ ]:
scores, results = get_scores_results_for_samples(random_samples, two_step_data)
best_index = scores.index(min(scores))
best = random_samples[best_index]

print(f"""The best parameter set is Set {best_index},
k1 = {best[0]}, 
k2 = {best[1]},
k3 = {best[2]}, 
resulting in a score of {min(scores)}.""")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,3))

ax[0].errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], c='k', label=r'$Y_1$', **plot_style_dict)
ax[1].errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], c='k', label=r'$Y_2$', **plot_style_dict)
ax[2].errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], c='k', label=r'$Y_3$', **plot_style_dict)

for axi in fig.axes:
    axi.set_xlabel('Time, min')
    axi.set_ylabel('Concentration, mM')
    axi.legend()
    
for i, res in enumerate(results):
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    color = m.to_rgba(i)
    ax[0].plot(res.t, res.y[0], c=color)
    ax[1].plot(res.t, res.y[1], c=color)
    ax[2].plot(res.t, res.y[2], c=color, label=i)
    
lines, labels = ax[2].get_legend_handles_labels()

fig.legend(lines[:-1], labels[:-1], title='Parameter Set', bbox_to_anchor=(1.05, 0.925), frameon=False)
plt.show()

### 4.2 Latin Hypercube Sampling

In some cases you realize that your uniformly chosen parameter set are actually pretty similar and you want to get a better coverage of the parameter space.
Here, Latin Hypercube Sampling comes in handy.
In latin hypercube sampling the range of each parameter is divided into $N$ equally probable intervals.
LHS ensures that for each parameter, each interval contains exactly one sample (see example below).

In [ ]:
from scipy.stats import qmc

In [ ]:
sampler = qmc.LatinHypercube(d=sample_dim)
lhs_sample_norm = sampler.random(n=n_samples) ## sampling is done

l_bounds = [lower_sample_boundary] * sample_dim
u_bounds = [higher_sample_boundary] * sample_dim

print(f'lower bounderies: {l_bounds}, upper bounderies: {u_bounds}')

lhs_sample = qmc.scale(lhs_sample_norm, l_bounds, u_bounds) ## stretch samples to bounderies

print(f'Latin hypercube samples: \n {lhs_sample}')

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(9, 6))

colors = np.arange(0, n_samples)

### plot parameter bounds
for axi in fig.axes:
    axi.plot([lower_sample_boundary, lower_sample_boundary], [higher_sample_boundary, lower_sample_boundary], color='k')
    axi.plot([higher_sample_boundary, lower_sample_boundary], [higher_sample_boundary, higher_sample_boundary], color='k')
    axi.plot([higher_sample_boundary, higher_sample_boundary], [lower_sample_boundary, higher_sample_boundary], color='k')
    axi.plot([lower_sample_boundary, higher_sample_boundary], [lower_sample_boundary, lower_sample_boundary], color='k')

### plot LHS grid
lhs_vals = np.linspace(lower_sample_boundary, higher_sample_boundary, n_samples+1)
for axi in fig.axes:
    for lhs_val in lhs_vals:
        axi.plot([lhs_val, lhs_val], [lower_sample_boundary, higher_sample_boundary], c='grey', alpha=0.3)
        axi.plot([lower_sample_boundary, higher_sample_boundary], [lhs_val, lhs_val], c='grey', alpha=0.3)

# plot random samples
    
ax[1][0].scatter(random_samples[:,0], random_samples[:,1], c='k', s=10)
ax[0][0].scatter(lhs_sample[:,0], lhs_sample[:,1], c=colors,cmap=cmap)


ax[1][1].scatter(random_samples[:,0], random_samples[:,2], c='k', s=10)
ax[0][1].scatter(lhs_sample[:,0], lhs_sample[:,2], c=colors,cmap=cmap)


ax[1][2].scatter(random_samples[:,1], random_samples[:,2], c='k', s=10, label='Uniform samples')
sc = ax[0][2].scatter(lhs_sample[:,1], lhs_sample[:,2], c=colors,cmap=cmap)



for i in range(2):
    ax[i][0].set_xlabel(r'$k_1$')
    ax[i][0].set_ylabel(r'$k_2$')


    ax[i][1].set_xlabel(r'$k_1$')
    ax[i][1].set_ylabel(r'$k_3$')

    ax[i][2].set_xlabel(r'$k_2$')
    ax[i][2].set_ylabel(r'$k_3$')
    


## add legend, adjust tick params etc.
fig.legend(*sc.legend_elements(), title='Parameter Set', bbox_to_anchor=(1.2, 0.95))


plt.tight_layout()
plt.show()

In most cases one should see that the LHS samples result in a better coverage of the parameter space.

In [ ]:
lhs_scores, lhs_results = get_scores_results_for_samples(lhs_sample, two_step_data)

best_index = lhs_scores.index(min(lhs_scores))
best = lhs_sample[best_index]

print(f"""The best parameter set is Set {best_index},
k1 = {best[0]}, 
k2 = {best[1]},
k3 = {best[2]}, 
resulting in a score of {min(lhs_scores)}.""")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,3))

ax[0].errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], fmt='o', capsize=2, markersize=3, c='k', label=r'$Y_1$')
ax[1].errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], fmt='o', capsize=2, markersize=3, c='k', label=r'$Y_2$')
ax[2].errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], fmt='o', capsize=2, markersize=3, c='k', label=r'$Y_3$')
for axi in fig.axes:
    axi.set_xlabel('Time, min')
    axi.set_ylabel('Concentration, mM')
    axi.legend()
    
for i, res in enumerate(lhs_results):
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    color = m.to_rgba(i)
    ax[0].plot(res.t, res.y[0], c=color)
    ax[1].plot(res.t, res.y[1], c=color)
    ax[2].plot(res.t, res.y[2], c=color, label=i)
    
lines, labels = ax[2].get_legend_handles_labels()
fig.legend(lines[:-1], labels[:-1], title='Parameter Set', bbox_to_anchor=(1.05, 0.925))
plt.show()

### 4.3 Logarithmic scale

In many cases you don't even now the magnitude of a parameter (i.e. if it is close to  $0.1$, $10$ or $1000$). In those cases it is beneficial to sample parameters logarithmically.
Furthermore, the effect of parameter changes on model results are often non-linear. A change of $k_1$ from $9$ to $10$ probably has a smaller effect than a change from $0.1$ to $1.1$.

In [ ]:
sampler = qmc.LatinHypercube(d=sample_dim)

# get lhs samples
lhs_log_sample = sampler.random(n=n_samples)

# transform boundaries to log scale
l_bounds = [np.log10(lower_sample_boundary)]*sample_dim
u_bounds = [np.log10(higher_sample_boundary)]*sample_dim

# scale samples according to logarithmic boundaries
lhs_log_sample = qmc.scale(lhs_log_sample, l_bounds, u_bounds)

# get parameter values from log LHS samples
lhs_log_sample = 10**(lhs_log_sample)

In [ ]:
lhs_log_scores, lhs_log_results = get_scores_results_for_samples(lhs_log_sample, two_step_data)

best_index = lhs_log_scores.index(min(lhs_log_scores))
lhs_log_best = lhs_log_sample[best_index]

print(f"""The best parameter set is Set {best_index},
k1 = {best[0]}, 
k2 = {best[1]},
k3 = {best[2]}, 
resulting in a score of {min(lhs_log_scores)}.""")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12,3))

ax[0].errorbar(two_step_data['Time'], two_step_data['y0_mean'], yerr=two_step_data['y0_std'], fmt='o', capsize=2, markersize=3, c='k', label=r'$Y_1$')
ax[1].errorbar(two_step_data['Time'], two_step_data['y1_mean'], yerr=two_step_data['y1_std'], fmt='o', capsize=2, markersize=3, c='k', label=r'$Y_2$')
ax[2].errorbar(two_step_data['Time'], two_step_data['y2_mean'], yerr=two_step_data['y2_std'], fmt='o', capsize=2, markersize=3, c='k', label=r'$Y_3$')
for axi in fig.axes:
    axi.set_xlabel('Time, min')
    axi.set_ylabel('Concentration, mM')
    axi.legend()
    
for i, res in enumerate(lhs_log_results):
    m = cm.ScalarMappable(norm=norm, cmap=cmap)
    color = m.to_rgba(i)
    ax[0].plot(res.t, res.y[0], c=color)
    ax[1].plot(res.t, res.y[1], c=color)
    ax[2].plot(res.t, res.y[2], c=color, label=i)
    
lines, labels = ax[2].get_legend_handles_labels()
fig.legend(lines[:-1], labels[:-1], title='Parameter Set', bbox_to_anchor=(1.05, 0.925))
plt.show()

Although the sampling helped you to get a better approximation of the parameter set, you are not at all satisfied.
Thus you look for local optimization algorithms which will help you to find the best fit.

## 5. Your turn






Now its your turn. You are given the dataset ``heinrich.csv`` and the ODE system:

$$
\frac{dy_0}{dt} = p_0 - y_0 - (p_1\cdot y_0)\cdot(1 + p_2\cdot y_1^4)
$$

$$
\frac{dy_1}{dt} = (p_1\cdot y_0)\cdot(1 + p_2\cdot y_1^4) - p_3\cdot y_1
$$

with the initial values $y_0 = 1$ and $y_1 = 0$. 
Find parameters which fit the data well.

## 5.1 Load the dataset

## 5.2 Define the model as python function

## 5.3 Define the cost function

## 5.4 Sample logarithmically 

To find a good initial parameter set for optimization.

## 5.5 Run the optimization 

WARNING: It might be neccessary to use every sampling result as input for the optimization, as the given model is not too easy to fit.
In this case you want to iterate over the samples and store the estimation results in a list to later check which one performed best.

# Appendix: Creation of Pseudo Data

## Heinrich Model

In [ ]:
y_init = [1, 0]
params = [6.77, 1.01, 1.26, 5.11]
res = solve_ivp(heinrich_model, (0, 10), y_init, args=(params,), t_eval=np.arange(0, 10, 0.1))

In [ ]:
reps = 6
th_point = 5

pseudo_time = res.t[::th_point]
psd_1 = []
psd_2 = []
for i in range(reps):
    pseudo_data_y1 = res.y[0,::th_point] + np.random.normal(0, res.y[0,::th_point]/10)
    pseudo_data_y2 = res.y[1,::th_point]+ np.random.normal(0, res.y[1,::th_point]/5)
    psd_1.append(pseudo_data_y1)
    psd_2.append(pseudo_data_y2)    

In [ ]:
psd_1_df = pd.DataFrame(psd_1).T
psd_2_df = pd.DataFrame(psd_2).T

In [ ]:
rep_cols = [f'Rep{i}' for i in range(reps)]

psd_1_df.columns = rep_cols
psd_1_df['mean'] = psd_1_df[rep_cols].mean(axis=1)
psd_1_df['std'] = psd_1_df[rep_cols].std(axis=1)


In [ ]:
rep_cols = [f'Rep{i}' for i in range(reps)]

psd_2_df.columns = rep_cols
psd_2_df['mean'] = psd_2_df[rep_cols].mean(axis=1)
psd_2_df['std'] = psd_2_df[rep_cols].std(axis=1)


In [ ]:
dataset = pd.DataFrame(columns=['Time', 'y0_mean', 'y0_std', 'y1_mean', 'y1_std'])
dataset['Time'] = pseudo_time
dataset['y0_mean'] = psd_1_df['mean']
dataset['y0_std'] = psd_1_df['std']
dataset['y1_mean'] = psd_2_df['mean']
dataset['y1_std'] = psd_2_df['std']

In [ ]:
dataset.to_csv('heinrich.csv', index=False)

## Two Step Model

In [ ]:
y_init = [10, 0.01, 0.01]
params = [0.3, 0.2, 2]
res = solve_ivp(two_step, (0, 20), y_init, args=(params,), t_eval=np.arange(0, 20, 0.1))

reps = 6
th_point = 10

pseudo_time = res.t[::th_point]
psd_1 = []
psd_2 = []
psd_3 = []
for i in range(reps):
    pseudo_data_y1 = res.y[0,::th_point] + np.random.normal(0,0.2+ res.y[0,::th_point]/20+ res.t[::th_point]/200)
    pseudo_data_y2 = res.y[1,::th_point]+ np.random.normal(0,0.4+ res.y[1,::th_point]/20+ res.t[::th_point]/200)
    pseudo_data_y3 = res.y[2,::th_point]+ np.random.normal(0,0.4+ res.y[2,::th_point]/20+ res.t[::th_point]/200)
    
    psd_1.append(pseudo_data_y1)
    psd_2.append(pseudo_data_y2) 
    psd_3.append(pseudo_data_y3)    
    
    
psd_1_df = pd.DataFrame(psd_1).T
psd_2_df = pd.DataFrame(psd_2).T
psd_3_df = pd.DataFrame(psd_3).T


rep_cols = [f'Rep{i}' for i in range(reps)]

psd_1_df.columns = rep_cols
psd_1_df['mean'] = psd_1_df[rep_cols].mean(axis=1)
psd_1_df['std'] = psd_1_df[rep_cols].std(axis=1)

psd_2_df.columns = rep_cols
psd_2_df['mean'] = psd_2_df[rep_cols].mean(axis=1)
psd_2_df['std'] = psd_2_df[rep_cols].std(axis=1)

psd_3_df.columns = rep_cols
psd_3_df['mean'] = psd_3_df[rep_cols].mean(axis=1)
psd_3_df['std'] = psd_3_df[rep_cols].std(axis=1)

dataset = pd.DataFrame(columns=['Time', 'y0_mean', 'y0_std', 'y1_mean', 'y1_std', 'y2_mean', 'y2_std'])
dataset['Time'] = pseudo_time
dataset['y0_mean'] = psd_1_df['mean']
dataset['y0_std'] = psd_1_df['std']
dataset['y1_mean'] = psd_2_df['mean']
dataset['y1_std'] = psd_2_df['std']
dataset['y2_mean'] = psd_3_df['mean']
dataset['y2_std'] = psd_3_df['std']

dataset.to_csv('two_step_dataset.csv', index=False)

fig, ax = plt.subplots()

ax.plot(res.t, res.y[0])
ax.plot(res.t, res.y[1])
ax.plot(res.t, res.y[2])


ax.errorbar(pseudo_time, psd_1_df['mean'], yerr=psd_1_df['std'], fmt='o', capsize=2, markersize=3, c='C0')
ax.errorbar(pseudo_time, psd_2_df['mean'], yerr=psd_2_df['std'], fmt='o', capsize=2, markersize=3, c='C1')
ax.errorbar(pseudo_time, psd_3_df['mean'], yerr=psd_3_df['std'], fmt='o', capsize=2, markersize=3, c='C2')


plt.show()